In [1]:
# create data : from sklearn.datasets import make_blobs 
#https://www.programcreek.com/python/example/100588/keras.layers.normalization.BatchNormalization

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
n_samples=100000
n_features=20
X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=5, n_redundant=2, n_repeated=0, n_classes=2,\
                           n_clusters_per_class=2, weights=[0.9,0.1], flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0,\
                           scale=1.0, shuffle=True, random_state=None)
y=np.array(y)
X=np.array(X)
y=np.reshape(y,(n_samples, 1))
X=np.reshape(X, (n_samples, n_features))

In [3]:

y.shape

(100000, 1)

In [22]:

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.noise import AlphaDropout
from keras.utils import np_utils
from keras.optimizers import RMSprop
from keras.initializers import lecun_uniform
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

from keras import backend as K
from keras import callbacks
from keras import layers
from keras import models
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from sklearn import pipeline
from sklearn import preprocessing
from sklearn.externals import joblib

def construct_model(layers=[50,50], activation='relu', b_init='random_uniform', k_init='random_uniform', dropout=.0,\
                    a_dropout=.0, optimizer='sgd', loss='binary_crossentropy', metrics='roc_auc_score',input_dim=20, output_dim=1, model_name='most'):
    """
    Helper to construct a Keras model based on dict of specs and input size

    Parameters
    ----------
    model_spec: dict
        Dict containing keys: arch, activation, dropout, optimizer, loss,
            w_reg, metrics
    input_dim: int
        Size of input dimension
    output_dim: int
        Size of input dimension

    Returns
    -------
    model: Compiled keras.models.Sequential

    """

    model = Sequential()

    for li, layer_size in enumerate(layers):
        # For input layer, add input dimension
        if li == 0:
            temp_input_dim = input_dim
            model.add(Dense(layer_size,
                            input_dim=temp_input_dim,
                            activation=activation,
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Input'))
        elif li < len(layers)-1:
            model.add(Dense(layer_size,
                            activation=activation,
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Layer_%i' % li))
        elif li == len(layers)-1:
            model.add(Dense( output_dim ,
                            activation='sigmoid',
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Layer_%i' % li))
            
        if dropout > 0. :
            if a_dropout == 0.:
                model.add(Dropout(dropout, name='Dropout_%i' % li))
        if a_dropout > 0.:
            model.add(Dropout(a_dropout, name='Dropout_%i' % li))
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[metrics],
                 loss_weights=[2])
    #keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, save_best_only=True, \
    #                                write_graph=True, write_images=False)

    return model 
#early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='auto')
pipe = pipeline.Pipeline([
    ('rescale', preprocessing.StandardScaler()),
    ('nn', KerasClassifier(build_fn=construct_model, epochs=10, batch_size=128,
                           validation_split=0.2))#, callbacks=[early_stopping]
])

SyntaxError: invalid syntax (<ipython-input-22-540ce05ef526>, line 26)

In [13]:
def construct_model_no_act(layers=[50,50], activation='relu', a_reg='l1', b_reg='l1', b_init='random_uniform', k_init='random_uniform', dropout=.0,\
                    a_dropout=.0, optimizer='sgd', loss='binary_crossentropy', metrics='accuracy',input_dim=20, output_dim=1, model_name='most'):
    """
    Helper to construct a Keras model based on dict of specs and input size

    Parameters
    ----------
    model_spec: dict
        Dict containing keys: arch, activation, dropout, optimizer, loss,
            w_reg, metrics
    input_dim: int
        Size of input dimension
    output_dim: int
        Size of input dimension

    Returns
    -------
    model: Compiled keras.models.Sequential

    """

    model = Sequential()

    for li, layer_size in enumerate(layers):
        # For input layer, add input dimension
        if li == 0:
            temp_input_dim = input_dim
            model.add(Dense(layer_size,
                            input_dim=temp_input_dim,
                            activation=activation,
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Input'))
        elif li < len(layers)-1:
            model.add(Dense(layer_size,
                            activation=activation,
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Layer_%i' % li))
        elif li == len(layers)-1:
            model.add(Dense( output_dim ,
                            activation=None,
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Layer_%i' % li))
            
        if dropout > 0.:
            model.add(Dropout(dropout, name='Dropout_%i' % li))
        if a_dropout > 0.:
            model.add(Dropout(a_dropout, name='Dropout_%i' % li))
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[metrics])
    #keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, save_best_only=True, \
    #                                write_graph=True, write_images=False)
    return model 
#early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='auto')
pipe2 = pipeline.Pipeline([
    ('rescale', preprocessing.StandardScaler()),
    ('nn', KerasClassifier(build_fn=construct_model_no_act, epochs=10, batch_size=128,
                           validation_split=0.2))#, callbacks=[early_stopping])
])

In [14]:
#custom scoring function sklearn
from sklearn.grid_search import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
def lift_index(y, probabilities):
    #rearrange probabilities and y in descending order
    probabilities = [val[0] for val in probabilities]
    print(probabilities)
    combine = list(zip(y, probabilities))
    dtype = [('data', int), ('prob', float)]
    combine = np.array(combine,dtype=dtype)
    combine = np.sort(combine, order='prob') #sort according to probabilities
    ordered = np.sort(probabilities) #sort probabilities
    #divide into deciles
    deciles = pd.qcut(ordered, 10, duplicates='drop', labels=False) #creates array of digits from 1 to 10
    #sum of positive in each decile-if negative then zero and doesn't get taken into account
    sum_deciles = sum([((val+1)/10)*combine[num][0] for num, val in enumerate(deciles)])
    return sum_deciles/sum(y)
lift_index_score = make_scorer(lift_index, greater_is_better=True, needs_proba=True )
    

In [15]:
y=[0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1]
probabilities = [0.6, 0.7,0.8,0.1,0.2,0.4,0.6,0.7,0.8,0.03,0.2, 0.4,0.2,0.9,0.2,0.75,0.243,0.5,0.75,0.03]
combine = list(zip(y, probabilities))
dtype = [('data', int), ('prob', float)]
combine = np.array(combine,dtype=dtype)
combine = np.sort(combine, order='prob') #sort according to probabilities
ordered = np.sort(probabilities) #sort probabilities
#divide into deciles
deciles = pd.qcut(ordered, 10, duplicates='drop',labels=False) #creates array of digits from 1 to 10
sum_deciles = sum([((val+1)/10)*combine[num][0] for num, val in enumerate(deciles)])
deciles

array([0, 0, 1, 1, 1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 8],
      dtype=int64)

In [16]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [17]:
import matplotlib.pyplot as plt
from matplotlib import cm

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [18]:
def plot_many_score(results, scoring):
    """
    This function prints and plots a graph conditioned on some variable that shows progress in accuracy.
    results is the cv_results_ attribute after calling gs callable.
    scoring is dict of scoring functions.
    """
    plt.figure(figsize=(13, 13))
    plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
              fontsize=16)
    
    plt.xlabel("min_samples_split")
    plt.ylabel("Score")
    plt.grid()
    
    ax = plt.axes()
    ax.set_xlim(0, 402)
    ax.set_ylim(0.73, 1)
    
    # Get the regular numpy array from the MaskedArray
    X_axis = np.array(results['param_min_samples_split'].data, dtype=float)
    
    for scorer, color in zip(sorted(scoring), ['g', 'k']):
        for sample, style in (('train', '--'), ('test', '-')):
            sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
            sample_score_std = results['std_%s_%s' % (sample, scorer)]
            ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                            sample_score_mean + sample_score_std,
                            alpha=0.1 if sample == 'test' else 0, color=color)
            ax.plot(X_axis, sample_score_mean, style, color=color,
                    alpha=1 if sample == 'test' else 0.7,
                    label="%s (%s)" % (scorer, sample))
    
        best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
        best_score = results['mean_test_%s' % scorer][best_index]
    
        # Plot a dotted vertical line at the best score for that scorer marked by x
        ax.plot([X_axis[best_index], ] * 2, [0, best_score],
                linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)
    
        # Annotate the best score for that scorer
        ax.annotate("%0.2f" % best_score,
                    (X_axis[best_index], best_score + 0.005))
    
    plt.legend(loc="best")
    plt.grid('off')
    plt.show()

In [21]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import itertools
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV,RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
#fix seed for reproducibility
seed=7
np.random.seed(seed)
n_samples=100000
n_features=20
X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=5, n_redundant=2, n_repeated=0, n_classes=2,\
                           n_clusters_per_class=2, weights=[0.9,0.1], flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0,\
                           scale=1.0, shuffle=True, random_state=None)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

#import data without any dummy variables or changes
param_grid = {'nn__layers': [[100,100],[100,100,100],[50,50,50,50]],
              'nn__activation':['selu'],
              'nn__dropout': [0.],
              'nn__a_dropout': [0.2],
              'nn__k_init': ['lecun_normal'],
              'nn__b_init': ['lecun_normal'],
              'nn__optimizer':['adam'],
              'nn__metrics' : ['accuracy'],
              'nn__epochs':[10],
              'nn__batch_size':[32]}

#improve with article
grid = GridSearchCV(pipe,
                    param_grid,
                    return_train_score=True,
                    refit=True,
                    scoring='roc_auc',
                    verbose=15)
rand_param_grid = {'nn__layers': [[100,100],[100,100,100],[50,50,50,50]],
              'nn__activation':['relu','selu'],
              'nn__dropout': scipy.stats.expon(scale=0.5),
              'nn__a_dropout': scipy.stats.expon(scale=0.2),
              'nn__k_init': ['lecun_normal'],
              'nn__b_init': ['lecun_normal'],
              'nn__optimizer':('rmsprop','adam'),
              'nn__metrics' : ['accuracy'],
              'nn__epochs':[10,50],
              'nn__batch_size':[32,16]} 
#improve with article
rand_grid = RandomizedSearchCV(pipe2, 
                               rand_param_grid,
                               return_train_score=True,
                               refit=True,
                               scoring= 'roc_auc')
grid_result = grid.fit(X_train,y_train) #reg search
grid_result_2 = rand_grid.fit(X_train,y_train)
#reg search
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
#random searh
means2 = grid_result_2.cv_results_['mean_test_score']
stds2 = grid_result_2.cv_results_['std_test_score']
params2 = grid_result_2.cv_results_['params']
for mean, stdev, param in zip(means2, stds2, params2):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


RuntimeError: Cannot clone object <keras.wrappers.scikit_learn.KerasClassifier object at 0x0000016FB0236F60>, as the constructor does not seem to set parameter callbacks

In [ ]:
#visualize best model
n_classes = 2
model_name = 'dense'
plot_confusion_matrix(confusion_matrix(grid_result.predict_classes(X_test), y_test), n_classes)

In [ ]:
model=grid_result #redundant variable change 
best = construct_model( layers=model.best_params_['layers'],
              activation=model.best_params_['activation'],
              b_init=model.best_params_['b_init'],
              k_init=model.best_params_['k_init'],
              dropout=model.best_params_['dropout'],
              a_dropout=model.best_params_['a_dropout'],
              optimizer=model.best_params_['optimizer'],
              metrics=model.best_params_['metrics'] )
plot_history(best.fit(X_train, y_train, validation_split=0.33, epochs=50, batch_size=10, verbose=10))

In [ ]:
#save model
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

save(best, "best model everrr")